<a href="https://colab.research.google.com/github/B132020005/B132020005/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_token'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, TemplateMessage,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction,ConfirmTemplate,MessageAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('YOUR_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('YOUR_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
          reply = TemplateMessage(
          alt_text="這是確認視窗",
          template=ConfirmTemplate(
             text="你喜歡看籃球嗎？",#the question
             actions=[
               MessageAction(label="是", text="我超喜歡"),#one of the botton
               MessageAction(label="否", text="我對其他運動比較有興趣")
          ]
        )
      )
        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Los_Angeles_Lakers_logo.svg/255px-Los_Angeles_Lakers_logo.svg.png',
                        title='洛杉磯湖人',
                        text='一支位於美國加利福尼亞州洛杉磯的職業籃球隊',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/wiki/%E6%B4%9B%E6%9D%89%E7%9F%B6%E6%B9%96%E4%BA%BA'),
                            MessageAction(label='投票', text="湖人總冠軍")
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/d/da/Golden_State_Warriors.png',
                        title='金洲勇士',
                        text='一支位於美國加利福尼亞州舊金山的美國職業籃球隊',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%87%91%E5%B7%9E%E5%8B%87%E5%A3%AB'),
                            MessageAction(label='投票', text="勇士總冠軍")
                        ]
                    )
                ]
            )
            # Corrected indentation and typo in TemplateMessage
            reply = TemplateMessage(
                    alt_text="這是輪播視窗",
                    template=carousel_template
                )
        else:
          reply = TextMessage(text='Thanks!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                   reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b6d1-34-145-112-11.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:02:30] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:02:30] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:02:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/May/2025 07:03:42] "POST /callback HTTP/1.1" 200 -
